# Suppliers Data Preprocessing

AIM : Here you need to transform the supplier data to achieve the same granularity as the target data. <br>
(Hint: how many rows per product do you have in the target data?).
Be aware of character encodings when processing the data.

## Data Profiling (before preprocessing)

In [715]:
import os, json
#json file path
fname = 'C:/Users/toami/Downloads/Data Analyst Remote Task/Data Analyst Remote Task/supplier_car.json'

In [716]:
#Reading all json objects from a file into a list
json_objects = []
for line in open(fname, 'r'):
    json_objects.append(json.loads(line))

In [717]:
#Taking Json data into a dataframe
import pandas as pd
df=pd.DataFrame()
for index in range (0,len(json_objects)):
    df1=pd.DataFrame([json_objects[index]])
    df = df.append(df1)
    
df.reset_index(drop=True, inplace=True)

In [718]:
df.head(5)

,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,Attribute Names,Attribute Values,entity_id
0,976.0,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,Seats,2,0001fda6-192b-46a8-bc08-0e833f904eed
1,1059.0,MERCEDES-BENZ,ML 350 Inspiration,MERCEDES-BENZ ML 350 Inspiration,ML 350,ML 350 Inspiration,Hp,235,00107c2d-0071-4475-88f0-810133638b7e
2,524.0,AUDI,S6 Avant quattro 4.2,AUDI S6 Avant quattro 4.2,S6,S6 Avant quattro 4.2,FuelTypeText,Benzin,00126794-a8ef-48fe-93d6-43cfc69fbfb6
3,608.0,SAAB,9-3 2.0i-16 TS Aero,SAAB 9-3 2.0i-16 TS Aero,9-3,9-3 2.0i-16 TS Aero,Ccm,1985,00182529-1bf7-4f93-89fa-2e8e634b2c9d
4,726.0,PORSCHE,911 Turbo Cabrio,PORSCHE 911 Turbo Cabrio,911,911 Turbo Cabrio,BodyColorText,schwarz mÃ©t.,002d30c2-43f6-4905-868f-160dbc445c56


In [719]:
#Checking number of rows and columns
df.shape

(21906, 9)

In [720]:
#Checking number of duplicate rows
df.duplicated(keep='first').sum()

0

In [721]:
df.columns

Index(['ID', 'MakeText', 'TypeName', 'TypeNameFull', 'ModelText',
       'ModelTypeText', 'Attribute Names', 'Attribute Values', 'entity_id'],
      dtype='object')

In [722]:
#Checking Data Consistency
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero_Values', 1 : 'Missing_Values', 2 : '%_of_Total_Values'})
        mz_table['Total_Zero_Missing_Values'] = mz_table['Zero_Values'] + mz_table['Missing_Values']
        mz_table['%_Total_Zero_Missing Values'] = 100 * mz_table['Total_Zero_Missing_Values'] / len(df)
        mz_table['Data_Type'] = df.dtypes
        mz_table = mz_table.sort_values('%_of_Total_Values', ascending=False).round(1)
            
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
              "There are " + 
               str(mz_table.Total_Zero_Missing_Values[mz_table['Total_Zero_Missing_Values'] >= 1.0].count()) +
              " columns that have zero and missing values." + " Rows.\n"
              "There are " + str(mz_table.Zero_Values[mz_table['Zero_Values'] >= 1.0].count()) +
              " columns that have zero values." + " Rows.\n"
              "There are " + str(mz_table.Missing_Values[mz_table['Missing_Values'] >= 1.0].count()) +
              " columns that have missing values."
              )
        return mz_table

missing_zero_values_table(df)

Your selected dataframe has 9 columns and 21906 Rows.
There are 1 columns that have zero and missing values. Rows.
There are 0 columns that have zero values. Rows.
There are 1 columns that have missing values.


,Zero_Values,Missing_Values,%_of_Total_Values,Total_Zero_Missing_Values,%_Total_Zero_Missing Values,Data_Type
ModelText,0,949,4.3,949,4.3,object
ID,0,0,0.0,0,0.0,object
MakeText,0,0,0.0,0,0.0,object
TypeName,0,0,0.0,0,0.0,object
TypeNameFull,0,0,0.0,0,0.0,object
ModelTypeText,0,0,0.0,0,0.0,object
Attribute Names,0,0,0.0,0,0.0,object
Attribute Values,0,0,0.0,0,0.0,object
entity_id,0,0,0.0,0,0.0,object


Since ModelText has only 4.3% of missing values which is very less and acceptable, we will not drop this column.

## Data Preprocessing

In [723]:
#ID uniquely identify a car
df.ID.nunique()

1153

In [724]:
#changing datatype of ID from string to float
df['ID'] = df['ID'].astype(float)

#sorting on ID
df.sort_values(by=['ID'], inplace=True)

In [725]:
#Many attributes are stacked in rows and need to be column form
df['Attribute Names'].unique()

array(['Km', 'InteriorColorText', 'BodyColorText', 'Properties',
       'ConsumptionTotalText', 'Hp', 'Ccm', 'BodyTypeText',
       'ConsumptionRatingText', 'Doors', 'Seats', 'ConditionTypeText',
       'TransmissionTypeText', 'FuelTypeText', 'FirstRegYear', 'City',
       'FirstRegMonth', 'Co2EmissionText', 'DriveTypeText'], dtype=object)

In [726]:
#Creating a pivoted table to get attribute names and their values as columns
df3=df[['ID', 'Attribute Names', 'Attribute Values']]
df3_pivoted = df.pivot(index="ID", columns="Attribute Names", values="Attribute Values")
df3_pivoted.reset_index(inplace=True)

In [727]:
df.drop(columns=['Attribute Names', 'Attribute Values', 'entity_id'], inplace=True)
df.drop_duplicates(inplace=True)

In [728]:
df_full = pd.merge(df, df3_pivoted, on="ID")
df_full.head(5)

,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,BodyColorText,BodyTypeText,Ccm,City,...,DriveTypeText,FirstRegMonth,FirstRegYear,FuelTypeText,Hp,InteriorColorText,Km,Properties,Seats,TransmissionTypeText
0,1.0,MERCEDES-BENZ,E 320 ElÃ©gance 4-Matic,MERCEDES-BENZ E 320 ElÃ©gance 4-Matic,E 320,E 320 ElÃ©gance 4-Matic,anthrazit,Limousine,3199,Zuzwil,...,Allrad,1,1999,Benzin,224,grau,31900,"""Ab MFK""",5,Automat
1,2.0,AUDI,RS6 Avant 5.0 V10 quattro,AUDI RS6 Avant 5.0 V10 quattro,RS6,RS6 Avant 5.0 V10 quattro,anthrazit,Kombi,4991,Zuzwil,...,Allrad,7,2008,Benzin,580,null,25400,"""Ab MFK""",5,Automat sequentiell
2,3.0,AUDI,RS6 Avant quattro,AUDI RS6 Avant quattro,RS6,RS6 Avant quattro,anthrazit,Kombi,4172,Zuzwil,...,Allrad,10,2002,Benzin,450,beige,38500,"""Ab MFK""",5,Automat sequentiell
3,4.0,CHEVROLET,Corvette Z06,CHEVROLET Corvette Z06,CORVETTE,Corvette Z06,anthrazit,CoupÃ©,6162,Zuzwil,...,Hinterradantrieb,6,2015,Benzin,660,schwarz,200,"""Ab MFK"", ""Direkt-/Parallelimport""",2,Schaltgetriebe manuell
4,5.0,PORSCHE,Cayenne Turbo Techart Magnum Kit,PORSCHE Cayenne Turbo Techart Magnum Kit,CAYENNE,Cayenne Turbo Techart Magnum Kit,anthrazit,SUV / GelÃ¤ndewagen,4806,Zuzwil,...,Allrad,1,2010,Benzin,500,schwarz,2900,"""Ab MFK""",5,Automat sequentiell


In [729]:
#Converting all numeric columns to int or float type accordingly
df_full['Ccm'] = df_full['Ccm'].astype(float)
df_full['Hp'] = df_full['Hp'].astype(float)
df_full['Km'] = df_full['Km'].astype(float)
df_full['FirstRegYear'] = df_full['FirstRegYear'].astype(int)
df_full['Seats'] = df_full['Seats'].astype(int)
df_full['FirstRegMonth'] = df_full['FirstRegMonth'].astype(int)
df_full['Doors'] = df_full['Doors'].astype(int)

In [730]:
#splittting numeric and text part of attribute Co2EmissionText
new = df_full["Co2EmissionText"].str.split(" ", n = 1, expand = True)
new = new.rename(columns = {0 : 'Co2Emission(g/km)', 1 : 'unit'})
new['Co2Emission(g/km)'] = pd.to_numeric(new['Co2Emission(g/km)'], errors='coerce')

#adding numeric part to the main dataframe
df_full['Co2Emission(g/km)'] = new['Co2Emission(g/km)']

#dropping old Co2EmissionText column
df_full.drop('Co2EmissionText', axis=1, inplace=True)

In [731]:
#splittting numeric and text part of attribute ConsumptionTotalText
new = df_full["ConsumptionTotalText"].str.split(" ", n = 1, expand = True)
new = new.rename(columns = {0 : 'Consumption(l/100km)', 1 : 'unit'})
new['Consumption(l/100km)'] = pd.to_numeric(new['Consumption(l/100km)'], errors='coerce')

#adding numeric part to the main dataframe
df_full['Consumption(l/100km)'] = new['Consumption(l/100km)']

#dropping old ConsumptionTotalText column
df_full.drop('ConsumptionTotalText', axis=1, inplace=True)

In [732]:
df_full.head(5)

,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,BodyColorText,BodyTypeText,Ccm,City,...,FirstRegYear,FuelTypeText,Hp,InteriorColorText,Km,Properties,Seats,TransmissionTypeText,Co2Emission(g/km),Consumption(l/100km)
0,1.0,MERCEDES-BENZ,E 320 ElÃ©gance 4-Matic,MERCEDES-BENZ E 320 ElÃ©gance 4-Matic,E 320,E 320 ElÃ©gance 4-Matic,anthrazit,Limousine,3199.0,Zuzwil,...,1999,Benzin,224.0,grau,31900.0,"""Ab MFK""",5,Automat,275.0,11.5
1,2.0,AUDI,RS6 Avant 5.0 V10 quattro,AUDI RS6 Avant 5.0 V10 quattro,RS6,RS6 Avant 5.0 V10 quattro,anthrazit,Kombi,4991.0,Zuzwil,...,2008,Benzin,580.0,null,25400.0,"""Ab MFK""",5,Automat sequentiell,333.0,14.0
2,3.0,AUDI,RS6 Avant quattro,AUDI RS6 Avant quattro,RS6,RS6 Avant quattro,anthrazit,Kombi,4172.0,Zuzwil,...,2002,Benzin,450.0,beige,38500.0,"""Ab MFK""",5,Automat sequentiell,350.0,14.6
3,4.0,CHEVROLET,Corvette Z06,CHEVROLET Corvette Z06,CORVETTE,Corvette Z06,anthrazit,CoupÃ©,6162.0,Zuzwil,...,2015,Benzin,660.0,schwarz,200.0,"""Ab MFK"", ""Direkt-/Parallelimport""",2,Schaltgetriebe manuell,291.0,12.7
4,5.0,PORSCHE,Cayenne Turbo Techart Magnum Kit,PORSCHE Cayenne Turbo Techart Magnum Kit,CAYENNE,Cayenne Turbo Techart Magnum Kit,anthrazit,SUV / GelÃ¤ndewagen,4806.0,Zuzwil,...,2010,Benzin,500.0,schwarz,2900.0,"""Ab MFK""",5,Automat sequentiell,270.0,11.5


## Data Profiling (After preprocessing)

### Structure Discovery

In [733]:
#Checking number of rows and columns
df_full.shape

(1153, 25)

In [734]:
df_full.columns

Index(['ID', 'MakeText', 'TypeName', 'TypeNameFull', 'ModelText',
       'ModelTypeText', 'BodyColorText', 'BodyTypeText', 'Ccm', 'City',
       'ConditionTypeText', 'ConsumptionRatingText', 'Doors', 'DriveTypeText',
       'FirstRegMonth', 'FirstRegYear', 'FuelTypeText', 'Hp',
       'InteriorColorText', 'Km', 'Properties', 'Seats',
       'TransmissionTypeText', 'Co2Emission(g/km)', 'Consumption(l/100km)'],
      dtype='object')

In [735]:
#Checking number of duplicate rows
df_full.duplicated(keep='first').sum()

0

In [736]:
#Checking Data Consistency
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero_Values', 1 : 'Missing_Values', 2 : '%_of_Total_Values'})
        mz_table['Total_Zero_Missing_Values'] = mz_table['Zero_Values'] + mz_table['Missing_Values']
        mz_table['%_Total_Zero_Missing Values'] = 100 * mz_table['Total_Zero_Missing_Values'] / len(df)
        mz_table['Data_Type'] = df.dtypes
        mz_table = mz_table.sort_values('%_of_Total_Values', ascending=False).round(1)
            
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
              "There are " + 
               str(mz_table.Total_Zero_Missing_Values[mz_table['Total_Zero_Missing_Values'] >= 1.0].count()) +
              " columns that have zero and missing values." + " Rows.\n"
              "There are " + str(mz_table.Zero_Values[mz_table['Zero_Values'] >= 1.0].count()) +
              " columns that have zero values." + " Rows.\n"
              "There are " + str(mz_table.Missing_Values[mz_table['Missing_Values'] >= 1.0].count()) +
              " columns that have missing values."
              )
        return mz_table

missing_zero_values_table(df_full)

Your selected dataframe has 25 columns and 1153 Rows.
There are 9 columns that have zero and missing values. Rows.
There are 5 columns that have zero values. Rows.
There are 4 columns that have missing values.


,Zero_Values,Missing_Values,%_of_Total_Values,Total_Zero_Missing_Values,%_Total_Zero_Missing Values,Data_Type
Co2Emission(g/km),0,341,29.6,341,29.6,float64
Consumption(l/100km),0,303,26.3,303,26.3,float64
ModelText,0,50,4.3,50,4.3,object
BodyTypeText,0,1,0.1,1,0.1,object
FirstRegMonth,3,0,0.0,3,0.3,int32
TransmissionTypeText,0,0,0.0,0,0.0,object
Seats,38,0,0.0,38,3.3,int32
Properties,0,0,0.0,0,0.0,object
Km,0,0,0.0,0,0.0,float64
InteriorColorText,0,0,0.0,0,0.0,object


In [738]:
df_full.head(5)

,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,BodyColorText,BodyTypeText,Ccm,City,...,FirstRegYear,FuelTypeText,Hp,InteriorColorText,Km,Properties,Seats,TransmissionTypeText,Co2Emission(g/km),Consumption(l/100km)
0,1.0,MERCEDES-BENZ,E 320 ElÃ©gance 4-Matic,MERCEDES-BENZ E 320 ElÃ©gance 4-Matic,E 320,E 320 ElÃ©gance 4-Matic,anthrazit,Limousine,3199.0,Zuzwil,...,1999,Benzin,224.0,grau,31900.0,"""Ab MFK""",5,Automat,275.0,11.5
1,2.0,AUDI,RS6 Avant 5.0 V10 quattro,AUDI RS6 Avant 5.0 V10 quattro,RS6,RS6 Avant 5.0 V10 quattro,anthrazit,Kombi,4991.0,Zuzwil,...,2008,Benzin,580.0,null,25400.0,"""Ab MFK""",5,Automat sequentiell,333.0,14.0
2,3.0,AUDI,RS6 Avant quattro,AUDI RS6 Avant quattro,RS6,RS6 Avant quattro,anthrazit,Kombi,4172.0,Zuzwil,...,2002,Benzin,450.0,beige,38500.0,"""Ab MFK""",5,Automat sequentiell,350.0,14.6
3,4.0,CHEVROLET,Corvette Z06,CHEVROLET Corvette Z06,CORVETTE,Corvette Z06,anthrazit,CoupÃ©,6162.0,Zuzwil,...,2015,Benzin,660.0,schwarz,200.0,"""Ab MFK"", ""Direkt-/Parallelimport""",2,Schaltgetriebe manuell,291.0,12.7
4,5.0,PORSCHE,Cayenne Turbo Techart Magnum Kit,PORSCHE Cayenne Turbo Techart Magnum Kit,CAYENNE,Cayenne Turbo Techart Magnum Kit,anthrazit,SUV / GelÃ¤ndewagen,4806.0,Zuzwil,...,2010,Benzin,500.0,schwarz,2900.0,"""Ab MFK""",5,Automat sequentiell,270.0,11.5


### Content discovery

In [741]:
#Checking count of unique values of all attributes
df_unique_count = pd.DataFrame(columns=['Attribute', 'Count of unique Values'])
for col in df_full.columns:
    df_unique_count = df_unique_count.append({
        'Attribute': col,
        'Count of unique Values': df_full[col].nunique()}, ignore_index=True)
    
df_unique_count

,Attribute,Count of unique Values
0,ID,1153
1,MakeText,70
2,TypeName,578
3,TypeNameFull,596
4,ModelText,290
5,ModelTypeText,596
6,BodyColorText,29
7,BodyTypeText,10
8,Ccm,273
9,City,6


In [742]:
#Checking min max values of all the numeric attributes
import numpy as np
Numerical_Columns = df_full.select_dtypes(np.number).columns.tolist()
df_numeric=df_full[Numerical_Columns]
df_min_max = pd.concat([df_numeric.min(), df_numeric.max()], axis=1)
df_min_max = df_min_max.rename(columns = {0 : 'Min_Values', 1 : 'Max_Values'})
df_min_max

,Min_Values,Max_Values
ID,1.0,1153.0
Ccm,0.0,8300.0
Doors,0.0,5.0
FirstRegMonth,0.0,12.0
FirstRegYear,1927.0,2016.0
Hp,0.0,1001.0
Km,1.0,316200.0
Seats,0.0,7.0
Co2Emission(g/km),127.0,575.0
Consumption(l/100km),5.1,24.9


In [743]:
df_full.columns

Index(['ID', 'MakeText', 'TypeName', 'TypeNameFull', 'ModelText',
       'ModelTypeText', 'BodyColorText', 'BodyTypeText', 'Ccm', 'City',
       'ConditionTypeText', 'ConsumptionRatingText', 'Doors', 'DriveTypeText',
       'FirstRegMonth', 'FirstRegYear', 'FuelTypeText', 'Hp',
       'InteriorColorText', 'Km', 'Properties', 'Seats',
       'TransmissionTypeText', 'Co2Emission(g/km)', 'Consumption(l/100km)'],
      dtype='object')

In [744]:
df_full['Consumption(l/100km)'].value_counts()

12.9    38
14.6    34
13.9    26
14.0    26
15.7    23
        ..
22.9     1
6.4      1
11.0     1
8.8      1
6.7      1
Name: Consumption(l/100km), Length: 130, dtype: int64

Analysing numerical attributes

# Normalisation

Normalisation is required in case an attribute value is different but actually is the same (different spelling, language, different unit used etc.). <br>
Example: if the source encodes Booleans as “Yes”/”No”, but the target as 1/0, then you would need
to normalise “Yes” to 1 and “No” to 0. <br>
Please normalise at least 2 attributes, and describe which normalisations are required for the other attributes including examples. <br>
• Input: pre-processed data <br>
• Output: normalised supplier data

In [745]:
df_full2=df_full.copy()

There are many character encodings in the dataframe. They are "Ã¼", 'Ã©', 'Ã¨', 'Ã‰' and 'Ã¤'. They will be replaced by their equivalents in the entire dataframe. They are replaced according to the UTF-8 Encoding Debugging Chart (https://www.i18nqa.com/debug/utf8-debug.html) <br>
"Ã¼" : "ü"<br>
'Ã©' : 'é'<br>
'Ã¨'  : 'è'<br>
'Ã‰' : 'É'<br>
'Ã¤' : 'ä'

In [746]:
#Replacing special encoders with equivalents
df_full2 = df_full2.replace('Ã¼','ü', regex=True)
df_full2 = df_full2.replace('Ã©','é', regex=True)
df_full2 = df_full2.replace('Ã¨','è', regex=True)
df_full2 = df_full2.replace('Ã‰','É', regex=True)
df_full2 = df_full2.replace('Ã¤','ä', regex=True)

I'll be normalising 'BodyTypeText', 'BodyColorText' and ConditionTypeText because they are present in the target data as well in the form of 'carType', 'color' and 'condition respectively. They are either completely or partially in German, they will be converted to English. <br>
Other attributes which need normalisation are 'ConditionTypeText', 'DriveTypeText', 'FuelTypeText', 'InteriorColorText', 'Properties', 'TransmissionTypeText' because all of them are either completely or partially in German langauge. So they need to be converted to english.

In [750]:
#Normalising BodyTypeText attribute
df_full2["BodyTypeText"].replace({"Kombi": "combi", 
                                  'Cabriolet' : 'convertible',
                            "Coupé": "Coupé",
                            "Kleinwagen": "small car", 
                            "Kompaktvan / Minivan": "compactvan / Minivan",
                            "Sattelschlepper": "articulated lorry",
                            "Wohnkabine": "Demountable Camper",
                            'SUV / Geländewagen': 'SUV'}, inplace=True)

In [751]:
#Normalising BodyColorText attribute
df_full2["BodyColorText"].replace({'anthrazit' : 'anthracite',
                            'anthrazit mét.' : 'anthracite met',
                            'beige': 'beige', 
                            'beige mét.' : 'beige met.',  
                            'blau' : 'blue',
                            'blau mét.' : 'blue met.',
                            'bordeaux' : 'bordeaux',
                            'bordeaux mét.' : 'bordeaux met.',
                            'braun' : 'brown',
                            'braun mét.' : 'brown met.',
                            'gelb': 'yellow',
                            'gelb mét.' : 'yellow met.',
                            'gold' : 'gold',
                            'gold mét.' : 'gold met.',
                            'grau' : 'Gray',
                            'grau mét.' : 'Gray met.',
                            'grün' : 'green',
                            'grün mét.' : 'green met.',
                            'orange' : 'orange',
                            'orange mét.' : 'orange met.',
                            'rot' : 'red',
                            'rot mét.' : 'red met.',
                            'schwarz' : 'black',
                            'schwarz mét.' : 'black met.',
                            'silber' : 'silver',
                            'silber mét.' : 'silver met.',
                            'violett mét.' : 'violet met.',
                            'weiss' : 'White',
                            'weiss mét.' : 'White met.'}, inplace=True)

In [752]:
#Normalising ConditionTypeText attribute
df_full2["ConditionTypeText"].replace({ 'Oldtimer' : 'Antique car',
                                       "Neu": "New",
                                       "Vorführmodell": "demonstration model"}, inplace=True)

In [753]:
df_full2.head(5)

,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,BodyColorText,BodyTypeText,Ccm,City,...,FirstRegYear,FuelTypeText,Hp,InteriorColorText,Km,Properties,Seats,TransmissionTypeText,Co2Emission(g/km),Consumption(l/100km)
0,1.0,MERCEDES-BENZ,E 320 Elégance 4-Matic,MERCEDES-BENZ E 320 Elégance 4-Matic,E 320,E 320 Elégance 4-Matic,anthracite,Limousine,3199.0,Zuzwil,...,1999,Benzin,224.0,grau,31900.0,"""Ab MFK""",5,Automat,275.0,11.5
1,2.0,AUDI,RS6 Avant 5.0 V10 quattro,AUDI RS6 Avant 5.0 V10 quattro,RS6,RS6 Avant 5.0 V10 quattro,anthracite,combi,4991.0,Zuzwil,...,2008,Benzin,580.0,null,25400.0,"""Ab MFK""",5,Automat sequentiell,333.0,14.0
2,3.0,AUDI,RS6 Avant quattro,AUDI RS6 Avant quattro,RS6,RS6 Avant quattro,anthracite,combi,4172.0,Zuzwil,...,2002,Benzin,450.0,beige,38500.0,"""Ab MFK""",5,Automat sequentiell,350.0,14.6
3,4.0,CHEVROLET,Corvette Z06,CHEVROLET Corvette Z06,CORVETTE,Corvette Z06,anthracite,Coupé,6162.0,Zuzwil,...,2015,Benzin,660.0,schwarz,200.0,"""Ab MFK"", ""Direkt-/Parallelimport""",2,Schaltgetriebe manuell,291.0,12.7
4,5.0,PORSCHE,Cayenne Turbo Techart Magnum Kit,PORSCHE Cayenne Turbo Techart Magnum Kit,CAYENNE,Cayenne Turbo Techart Magnum Kit,anthracite,SUV,4806.0,Zuzwil,...,2010,Benzin,500.0,schwarz,2900.0,"""Ab MFK""",5,Automat sequentiell,270.0,11.5


# Integration

Data Integration is to transform the supplier data with a specific data schema into a new dataset
with target data schema, such as to: <br>
• keep any attributes that you can map to the target schema<br>
• discard attributes not mapped to the target schema<br>
• keep the number of records as unchanged<br>
• Input: normalised supplier data<br>
• Output: integrated supplier data

In [754]:
#Loading target data
df2 = pd.read_excel('C:/Users/toami/Downloads/Data Analyst Remote Task/Data Analyst Remote Task/Target Data.xlsx')

In [755]:
df2.columns

Index(['carType', 'color', 'condition', 'currency', 'drive', 'city', 'country',
       'make', 'manufacture_year', 'mileage', 'mileage_unit', 'model',
       'model_variant', 'price_on_request', 'type', 'zip', 'manufacture_month',
       'fuel_consumption_unit'],
      dtype='object')

In [756]:
df_full3.columns

Index(['make', 'model', 'model_variant', 'color', 'carType', 'city',
       'condition', 'manufacture_month', 'manufacture_year', 'mileage',
       'type'],
      dtype='object')

**ATTRIBUTE MAPPING**

Output data attributes which are also present in supplier data with same or different names:<br>

| Output Data Attribute      | Preprocessed supplier Data attributes |
| ----------- | ----------- |
| city      | City       |
| make   | MakeText        |
| manufacture_year      | FirstRegYear       |
| mileage   | Km        |
| model      | ModelText       |
| model_variant   | ModelTypeText        |
| manufacture_month      | FirstRegMonth       |
|carType |	BodyTypeText |
|color	  |    BodyColorText |
|condition |	ConditionTypeText |


Attributes missing in supplier data, they will be marked as NA:<br>

| Attribute |
| ----------|
| currency  |
| drive   |
| price_on_request |
| zip   |
| fuel_consumption_unit      |

Attributes that needs to be hardcoded based on the reasons mentioned:<br>

| Target Data Attribute | Value | Reason for hardcoding |
| ----------- | ----------- | ----------- |
| country | CH | All cities in input data are from Switzerland |
| mileage_unit | Km | mileage atribute unit is km in input data |


Addition of new column 'type' in Input data to map against 'type' column in target data. In this column, values will be assigned on the basis of 'BodyTypeText' values. A table research was conducted to decide what categories these 'BodyTypeText' vehicles belong to.  Find values in table below: 

| 'BodyTypeText' value      | 'type' value |
| ----------- | ----------- |
| Limousine | car |
| combi   | car        |
| Coupé      | car       |
| SUV / cross-country   | car        |
| Demountable Camper      | Camper       |
| small car   | car        |
| compactvan / Minivan      | Van       |
| articulated lorry   | Truck        |
| Pick-up      | Truck       |

In [757]:
df_full3=df_full2.copy()

In [706]:
#Adding new 'Type' column in supplier data: 
df_full3['type'] = df_full3.apply(lambda row: 
                                  'Truck' if row['BodyTypeText'] == 'articulated lorry' or row['BodyTypeText'] == 'Pick-up' 
                                  else( 'Van' if row['BodyTypeText'] == 'compactvan / Minivan' 
                                       else( 'Camper' if row['BodyTypeText'] == 'Demountable Camper' else 'car')), axis=1)

In [758]:
#Adding country column in supplier data with value as 'CH'
df_full3['country'] = 'CH'

#Adding mileage_unit column in supplier data with value as 'Km'
df_full3['mileage_unit'] = 'Km'

In [759]:
#renaming column names of normalised supplier data to ease joining dataframes
df_full3.rename(columns={'City': 'city', 
                   'MakeText': 'make',
                   'FirstRegYear': 'manufacture_year', 
                   'Km': 'mileage', 
                   'ModelText': 'model', 
                   'ModelTypeText': 'model_variant',
                   'FirstRegMonth': 'manufacture_month',
                   'BodyTypeText' : 'carType',
                   'BodyColorText' : 'color',
                   'ConditionTypeText':'condition'}, inplace=True)

In [760]:
#deleting columns from supplier data which are absent in target data
df_full3.drop(["ID","TypeName","TypeNameFull","Ccm", 'Co2Emission(g/km)', 'ConsumptionRatingText',
              'Consumption(l/100km)', 'Doors', 'DriveTypeText', 'FuelTypeText','Hp', 
              'InteriorColorText', 'Properties', 'Seats','TransmissionTypeText' ],axis=1, 
              inplace = True)

In [761]:
#Appending integration ready supplier data with target data
df_merged = df2.append(df_full3, ignore_index=True)

In [762]:
df_merged.shape

(8405, 18)

In [763]:
df_merged.head(5)

,carType,color,condition,currency,drive,city,country,make,manufacture_year,mileage,mileage_unit,model,model_variant,price_on_request,type,zip,manufacture_month,fuel_consumption_unit
0,Convertible / Roadster,White,Used,USD,LHD,Zuzwil,CH,Toyota,1983,0.0,kilometer,Celica,2800 i Supra,False,car,NaN,3.0,l_km_consumption
1,Convertible / Roadster,Other,Original Condition,USD,LHD,London,GB,Ferrari,2001,0.0,mile,550,NaN,True,car,NaN,NaN,NaN
2,Convertible / Roadster,White,Used,USD,LHD,Scotts Valley,US,Porsche,1989,0.0,mile,"911 ""G""",NaN,True,car,NaN,NaN,NaN
3,Convertible / Roadster,Blue,Used,USD,LHD,London,GB,Rolls-Royce,1961,0.0,kilometer,Silver Cloud II,NaN,False,car,NaN,NaN,NaN
4,Convertible / Roadster,Black,Original Condition,USD,LHD,Hong Kong,HK,Gumpert,2010,0.0,kilometer,Apollo,Apollo S 800hp,False,car,NaN,NaN,NaN
